In [1]:
using CSV, JuMP

avg_demand = CSV.read("AverageDemandForecast.csv")

lb_demand = CSV.read("LowerBoundDemandForecast.csv")

ub_demand = CSV.read("UpperBoundDemandForecast.csv")

old_max_prod = CSV.read("MaxProductionCapacity.csv")
max_prod = old_max_prod[1:34,:] #clip useless last row

weeks = names(avg_demand)[2:length(names(avg_demand))] #keys are Symbols not int so be careful!

"""
prod_dict maps product name (str from csv) to a dict w/keys:
"biweekly_replenishment" (in pallets)
"prod_capac" (pallets)
"units_per_pallet" (pallets)
"avg_dem_week_x" average demand on week x for x in weeks array (skips so be careful!)
"lb_dem_week_x" lower bound demand forecast on week x for x in weeks array
"ub_dem_week_x" upper bound demand forecast on week x for x in weeks array
"type" string, what category of product
"late_cost", late cost per unit of a certain product (dependent on its category)

NOTE1:
Get lower bound demand forecast for "Pampers Diapers" on week 51:
>> prod_dict["Pampers Diapers"][string("lb_dem_week_51")]

Get lower bound demand forecast for "Pampers Diapers" on week x:
>> prod_dict["Pampers Diapers"][string("lb_dem_week_", x)]
"""
#late order cost per unit for diff product types = 2x retail val
late_cost = Dict("baby" => 100, "fabric" => 40, "family"=> 20, "fem" => 15, "grooming" => 40, "hair" => 16, "home" => 30, "oral" => 90, "PHC" => 20, "skin/personal" => 80)

prod_dict = Dict(max_prod[k,1] => Dict("biweekly_replenishment" => max_prod[k,2], "prod_capac" => max_prod[k,3], "units_per_pallet" => max_prod[k, 4], "type" => "") for k in 1:length(max_prod[:,1]))

for i in 1:length(max_prod[:,1])
    prodname = max_prod[i,1]
    for j in weeks
        prod_dict[prodname][string("avg_dem_week_", j)] = avg_demand[i,j]
        prod_dict[prodname][string("lb_dem_week_", j)] = lb_demand[i,j]
        prod_dict[prodname][string("ub_dem_week_", j)] = ub_demand[i,j]
        if i in 1:2
            prod_dict[prodname]["type"] = "baby"
        elseif i in 3:6
            prod_dict[prodname]["type"] = "fabric"
        elseif i in 7:9
            prod_dict[prodname]["type"] = "family"
        elseif i in 10:11
            prod_dict[prodname]["type"] = "fem"
        elseif i in 12:15
            prod_dict[prodname]["type"] = "grooming"
        elseif i in 16:19
            prod_dict[prodname]["type"] = "hair"
        elseif i in 20:24
            prod_dict[prodname]["type"] = "home"
        elseif i in 25:28
            prod_dict[prodname]["type"] = "oral"
        elseif i in 29:30
            prod_dict[prodname]["type"] = "PHC"
        elseif i in 31:34
            prod_dict[prodname]["type"] = "skin/personal"
        end
        prod_dict[prodname]["late_cost"] = late_cost[prod_dict[prodname]["type"]]
    end
end

#inventory cost (units per day)
inv_cost = 0.5

#max capacity of distrib center (pallets)
distrib_max = 7200

#production costs to weeks notice
prod_notice = [1, 2, 3, 4, 5]
rush_cost = Dict(5 => 0.5, 4 => 1, 3 => 2.50, 2 => 7.50, 1 => 25)

infinity = 1000000


┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1273
┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1273


thread = 1 warning: only found 3 / 4 columns on data row: 35. Filling remaining columns with `missing`


1000000

In [2]:
using Random,Distributions

#calculates sDev based on 95% conf interval, creates a normal distribution, returns a sample from it
function genDemand(lb,ub,avg)
    sig = max(avg-lb,ub-avg)/1.96
    d = Normal(avg,sig)
    round(first(rand(d, 1)))#pulls random number from dist, first() removes the number from a 1 element array, then it's rounded
end

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1273


genDemand (generic function with 1 method)

In [3]:
#Creating Test demand matrix T. For product i on week j, T[i,j] units were demanded
T = zeros((34, 52))
sigma = zeros((34,52))
for i in 1:34
    prodname = max_prod[i,1]
    for j in 1:52
        week = j - ((j-1)%2) #if j is odd, week = j; otherwise, week = j-1
        avg = prod_dict[prodname][string("avg_dem_week_", week)]
        lb = prod_dict[prodname][string("lb_dem_week_", week)]
        ub = prod_dict[prodname][string("ub_dem_week_", week)]
        sigma[i,j] = max(avg-lb,ub-avg)/1.96
        T[i,j] = genDemand(lb,ub,avg)/2
    end
end

In [453]:
#The goal is to have before each week the avg demand + alpha standard deviations extra.
#The optimizer will find the optimal alpha

In [4]:
#using CSV, JuMP, Cbc

#m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))
alpha = 1


incoming = zeros((34,52))
left = zeros((34,52))
failedCost = 0
holdingCost = 0
orderCost = 0

0

In [7]:
dem = avg_demand[1:34,2:27]
avgDem = zeros((34,52))
safetyNeeds = zeros(34,52)
for i in 1:34
    for j in 1:26
        avgDem[i,(2*j)-1] = dem[i,j]
        avgDem[i,(2*j)] = dem[i,j]
    end
end
avgDem = 0.5*avgDem

poop = 0
for i in 1:34
    for j in 1:52
        safetyNeeds[i,j] = round(avgDem[i,j] + alpha*sigma[i,j])
        poop = poop + 3.5*round(alpha*sigma[i,j])
    end
end



In [8]:
poop

5.0084328e7

In [492]:
#creates replenish vector and product limit vector
replenish = zeros(34)
productLimit = zeros(34)
unitsPerPallet = max_prod[:,4]
pallets = max_prod[:,2]
palletLimit = max_prod[:,3]
for i in 1:34
    replenish[i] = pallets[i]*unitsPerPallet[i]
    productLimit[i] = palletLimit[i]*unitsPerPallet[i]
end

In [493]:
function relu(vector)
    for i in 1:length(vector)
        if vector[i] < 0
            vector[i] = 0
        end
    end
end

relu (generic function with 1 method)

In [494]:
#masterLoop
for week in 1:51
    #assuming that at the beginning of week, we know demand for the week
    if week == 1
        begInv = incoming[:,week]
    elseif week == 2
        failedCost = 0
        begInv = left[:,week-1] + incoming[:,week]
    else
        begInv = left[:,week-1] + incoming[:,week]
    end
    leftover = begInv - T[:,week]
    
    #failed cost
    for i in 1:34
        if leftover[i] <0
            failedCost =  failedCost - leftover[i]*late_cost[prod_dict[max_prod[i,1]]["type"]]
        end
    end
    
    relu(leftover) #mutates leftover to make each entry nonnegative
    left[:,week] = leftover
    
    
    #cost of holding inventory
    for i in 1:34
        holdingCost = holdingCost + 3.50*left[i,week]
    end
    
    
    
    #needed = demand - left
    neededNow = safetyNeeds[:,week+1] - left[:,week]  
    orderedNow = zeros(34)
    for i in 1:34
        if (safetyNeeds[i,week+1] - left[i,week] - incoming[i,week+1] > 0)  
            #max production capacity constraint
            orderedNow[i] = min(safetyNeeds[i,week+1] - left[i,week] - incoming[i,week+1], replenish[i])
            incoming[i,week+1] = orderedNow[i] + incoming[i,week+1]
            orderCost = orderCost + orderedNow[i]*rush_cost[1] #cost of ordering
        end
    end
        
    orderedForLater = zeros(34)
    for j in 2:(52-week) #j represents the number of weeks beyond the current week
        for i in 1:34
            needed = avgDem[i,week+j]-incoming[i,week+j]
            if (needed > 0)
                orderedForLater[i] = min(needed, replenish[i] - orderedNow[i])#max production capacity constraint
                incoming[i,week+j] = incoming[i,week+j] + orderedForLater[i]
                if j <= 5
                    orderCost = orderCost + orderedForLater[i]*rush_cost[j] #cost of ordering
                else
                    orderCost = orderCost + orderedForLater[i]*rush_cost[5]
                end
                orderedNow[i] = orderedNow[i] + orderedForLater[i]
            end
        end
    end

    
    
end

In [495]:
println(failedCost + holdingCost + orderCost)

1.65398704e8


In [496]:
println(failedCost)

2.5150925e7


In [497]:
println(holdingCost)

7.27381585e7


In [498]:
println(orderCost)

6.75096205e7
